In [1]:
include("LiPoSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5

using Statistics

In [7]:
dodeca_10_states = ["D"*string(n) for n=1:10];
basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

"DATA/ALL_GAMMAS_B4_D10.h5"

In [11]:
println(" Initial state of Spin boson data")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

init_states_data_file = "SB_simulation_init_states.h5"

init_states_by_gamma = []
init_times_by_gamma = []

for γᵢ in γ

    println()
    println("γ =  "*γᵢ)

    h5open(init_states_data_file,"cw") do fid
        γ_group = create_group(fid, γᵢ) # open gamma coupling group
    end

    init_states = []
    init_times = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        tₛ, ρₛ = LiPoSID.read_timeevolution(evol_data_file_name, state, γᵢ)

        ρₒ = ρₛ[1]
        tₒ = tₛ[1]

        h5open(init_states_data_file,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open gamma coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["rho_init"] = convert.(ComplexF64, ρₒ)
            init_state_group["time_init"] = convert.(ComplexF64, tₒ)
        end
    
        push!(init_states, ρₒ)
        push!(init_times, tₒ)
    
    end

    push!(init_states_by_gamma, init_states)
    push!(init_times_by_gamma, init_times)

end

 Initial state of Spin boson data

γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 

In [ ]:
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)